In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

import pyspark.sql.types as T
import pyspark.sql.functions as F

from datetime import datetime

conf = SparkConf()
conf.setMaster("local").setAppName('My app')

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)


print('Запущен Spark версии', spark.version)

22/12/13 11:26:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Запущен Spark версии 3.0.0


# Создайте схему будущего фрейма данных. Схема должна включать следующие атрибуты:
* id -  уникальный идентификатор посетителя сайта. Тип – последовательность чисел фиксированной длины. Данное поле не является первичным ключом.
* timestamp – дата и время события в формате unix timestamp.
* type – тип события, значение из списка (факт посещения(visit), клик по визуальному элементу страницы(click), скролл(scroll), перед на другую страницу(move)).
* page_id – id текущей страницы. Тип - последовательность чисел фиксированной длины.
* tag – каждая страница с новостью размечается редакцией специальными тегами, которые отражают тематику конкретной новости со страницы. Возможный список тематик: политика, спорт, медицина и т.д.
* sign – наличие у пользователя личного кабинета. Значения – True/False.

In [2]:
data_schema = T.StructType([
    T.StructField('id', T.IntegerType(), True),
    T.StructField('timestamp', T.IntegerType(), True),
    T.StructField('type', T.StringType(), True),
    T.StructField('page_id', T.IntegerType(), True),
    T.StructField('tag', T.StringType(), True),
    T.StructField('sign', T.BooleanType(), True)
])

# Создайте датафрейм с описанной выше схемой данных и наполните датафрейм данными

In [3]:
data_row = [(12345, 1667627426, 'click', 101, 'Sport', False),
            (12346, 1667627328, 'scroll', 101, 'Sport', True),
            (12347, 1657627426, 'move', 101, 'Sport', False), 
            (12348, 1666627426, 'visit', 101, 'Sport', True), 
            (12349, 1667527426, 'click', 102, 'News', True),
            (12350, 1667617426, 'scroll', 102, 'News', False),
            (12351, 1667626426, 'move', 103, 'Music', False),
            (12352, 1667627326, 'visit', 103, 'Music', True),
            (12353, 1667628426, 'click', 104, 'Medicine', False),
            (12354, 1667637426, 'scroll', 104, 'Medicine', True),
            (12348, 1668627426, 'move', 105, 'Science', False),
            (12345, 1677627426, 'visit', 106, 'Science', True),
            (12346, 1567627326, 'scroll', 101, 'Sport', True),
            (12347, 1657627526, 'visit', 101, 'Sport', False), 
            (12348, 1666627626, 'move', 101, 'Sport', True), 
            (12349, 1667527726, 'scroll', 102, 'News', True),
            (12350, 1667617826, 'click', 102, 'News', False),
            (12351, 1637626926, 'visit', 103, 'Music', False),
            (12352, 1663625326, 'move', 103, 'Music', True),
            (12353, 1663648426, 'scroll', 104, 'Medicine', False),
            (12348, 1667337426, 'click', 104, 'Medicine', True),
            (12353, 1664624426, 'click', 105, 'Science', False),
            (12356, 1673629426, 'click', 106, 'Science', True)
           ]

In [4]:
data = spark.createDataFrame(data=data_row,schema=data_schema)

In [5]:
data = data.select(*[i for i in data.columns],
                 F.from_unixtime("timestamp").alias("time_ymd"))

In [6]:
data.show(50)

+-----+----------+------+-------+--------+-----+-------------------+
|   id| timestamp|  type|page_id|     tag| sign|           time_ymd|
+-----+----------+------+-------+--------+-----+-------------------+
|12345|1667627426| click|    101|   Sport|false|2022-11-05 05:50:26|
|12346|1667627328|scroll|    101|   Sport| true|2022-11-05 05:48:48|
|12347|1657627426|  move|    101|   Sport|false|2022-07-12 12:03:46|
|12348|1666627426| visit|    101|   Sport| true|2022-10-24 16:03:46|
|12349|1667527426| click|    102|    News| true|2022-11-04 02:03:46|
|12350|1667617426|scroll|    102|    News|false|2022-11-05 03:03:46|
|12351|1667626426|  move|    103|   Music|false|2022-11-05 05:33:46|
|12352|1667627326| visit|    103|   Music| true|2022-11-05 05:48:46|
|12353|1667628426| click|    104|Medicine|false|2022-11-05 06:07:06|
|12354|1667637426|scroll|    104|Medicine| true|2022-11-05 08:37:06|
|12348|1668627426|  move|    105| Science|false|2022-11-16 19:37:06|
|12345|1677627426| visit|    106| 

# Решите следующие задачи:

##  Вывести топ-5 самых активных посетителей сайта

In [7]:
data.groupby("id").count().orderBy("count",ascending=False).show(5)

+-----+-----+
|   id|count|
+-----+-----+
|12348|    4|
|12353|    3|
|12347|    2|
|12349|    2|
|12346|    2|
+-----+-----+
only showing top 5 rows



## Посчитать процент посетителей, у которых есть ЛК

In [8]:
1-(data.count() - data.filter(data.sign==True).count())/data.count()

0.5217391304347826

In [9]:
data.filter(data.sign==True).count()/data.count()*100

52.17391304347826

 ## Вывести топ-5 страниц сайта по показателю общего кол-ва кликов на данной странице

In [10]:
data.filter(data.type=="click").groupBy("page_id").count().orderBy("count",ascending=False).show(5)

+-------+-----+
|page_id|count|
+-------+-----+
|    102|    2|
|    104|    2|
|    101|    1|
|    106|    1|
|    105|    1|
+-------+-----+



## Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)

In [11]:
data = data.withColumn("time_category",F.floor(F.hour("time_ymd") / F.lit(4)))
data.show(50)

+-----+----------+------+-------+--------+-----+-------------------+-------------+
|   id| timestamp|  type|page_id|     tag| sign|           time_ymd|time_category|
+-----+----------+------+-------+--------+-----+-------------------+-------------+
|12345|1667627426| click|    101|   Sport|false|2022-11-05 05:50:26|            1|
|12346|1667627328|scroll|    101|   Sport| true|2022-11-05 05:48:48|            1|
|12347|1657627426|  move|    101|   Sport|false|2022-07-12 12:03:46|            3|
|12348|1666627426| visit|    101|   Sport| true|2022-10-24 16:03:46|            4|
|12349|1667527426| click|    102|    News| true|2022-11-04 02:03:46|            0|
|12350|1667617426|scroll|    102|    News|false|2022-11-05 03:03:46|            0|
|12351|1667626426|  move|    103|   Music|false|2022-11-05 05:33:46|            1|
|12352|1667627326| visit|    103|   Music| true|2022-11-05 05:48:46|            1|
|12353|1667628426| click|    104|Medicine|false|2022-11-05 06:07:06|            1|
|123

## Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.

In [12]:
data.groupby("time_category").count().orderBy("count",ascending=False).first()

Row(time_category=1, count=6)

##  Создайте второй фрейм данных, который будет содержать информацию о ЛК посетителя сайта со следующим списком атрибутов

1. Id – уникальный идентификатор личного кабинета
2. User_id – уникальный идентификатор посетителя
3. ФИО посетителя
4. Дату рождения посетителя 
5. Дата создания ЛК

Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.

In [13]:
data_schema_2 = T.StructType([
    T.StructField("id", T.IntegerType(),True),
    T.StructField("user_id", T.IntegerType(),True),
    T.StructField("name", T.StringType(),True),
    T.StructField("birthday", T.DateType(),True),
    T.StructField("data_create", T.DateType(),True),
])

In [14]:
data_row_2 = [
    (1, 12348, 'Иванов Иван Иванович', datetime.strptime("1970-01-01", "%Y-%m-%d"),datetime.strptime("2022-10-15", "%Y-%m-%d")),
    (2, 12353, 'Петров Петр Петрович', datetime.strptime("1980-02-02", "%Y-%m-%d"),datetime.strptime("2022-10-15", "%Y-%m-%d")),
    (3, 12347, 'Семенов Семен Семенович', datetime.strptime("1990-03-13", "%Y-%m-%d"),datetime.strptime("2022-10-16", "%Y-%m-%d"))
]

In [15]:
data_pc = spark.createDataFrame(data = data_row_2, schema = data_schema_2)
data_pc.show(5)

+---+-------+--------------------+----------+-----------+
| id|user_id|                name|  birthday|data_create|
+---+-------+--------------------+----------+-----------+
|  1|  12348|Иванов Иван Иванович|1970-01-01| 2022-10-15|
|  2|  12353|Петров Петр Петрович|1980-02-02| 2022-10-15|
|  3|  12347|Семенов Семен Сем...|1990-03-13| 2022-10-16|
+---+-------+--------------------+----------+-----------+

